In [1]:
! pip install tqdm
! pip install opencv-python
! pip install scipy

You are using pip version 9.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
from tqdm import tqdm_notebook as tqdm
import json
import os
from os.path import isfile, join
from os import listdir
from os.path import isfile, join
import numpy as np
import pickle
import faiss
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20,10)
import pandas as pd
import traceback
import scipy.ndimage
import itertools, operator
import datetime
import math

In [3]:
files = [f for f in os.listdir('videos') if isfile(join('videos', f))]

shows = {}
for file in files:
    fullname = file.split('/')[-1]
    fn = fullname.replace('_VOD','').replace('_high','')
    fn_split = fn.split('_')
    
    name = "_".join(fn_split[1:-2])
    season = fn_split[-2]
    episode = fn_split[-1].split('.')[0]
    
#     print("name: {}, season: {}, episode: {}".format(name, season, episode))
    
    if name not in shows:
        shows[name] = {season : {episode : fullname}}
    
    if season not in shows[name]:
        shows[name][season] = {episode : fullname}
        
    if episode not in shows[name][season]:
        shows[name][season][episode] = fullname

    
print(json.dumps(shows, indent=2))
print("\n Total of {} files".format(len(files)))

{
  "Divorce": {
    "S3": {
      "E1": "217394H1_Divorce_S3_E1.mxf.mp4",
      "E3": "217396H1_Divorce_S3_E3.mxf.mp4",
      "E2": "217395H1_Divorce_S3_E2.mxf.mp4"
    },
    "S2": {
      "E2": "208888H1_Divorce_S2_E2.mxf.mp4",
      "E3": "208889H1_Divorce_S2_E3.mxf.mp4",
      "E1": "208887H1_Divorce_S2_E1.mxf.mp4"
    },
    "S1": {
      "E1": "193757H1_Divorce_S1_E1.mxf.mp4",
      "E3": "193917H1_Divorce_S1_E3.mxf.mp4",
      "E2": "193916H1_Divorce_S1_E2.mxf.mp4"
    }
  },
  "Expeditie_Robinson": {
    "S11": {
      "E2": "226558H1_Expeditie_Robinson_S11_E2.mxf.mp4",
      "E3": "226559H1_Expeditie_Robinson_S11_E3.mxf.mp4",
      "E1": "226557H1_Expeditie_Robinson_S11_E1.mxf.mp4"
    },
    "S13": {
      "E1": "251218H1_Expeditie_Robinson_S13_E1.mxf.mp4",
      "E2": "250463H1_Expeditie_Robinson_S13_E2.mxf.mp4"
    },
    "S14": {
      "E3": "260888H1_Expeditie_Robinson_S14_E3.mxf.mp4",
      "E1": "262135H1_Expeditie_Robinson_S14_E1.mxf.mp4",
      "E2": "262136H1_Expedi

In [4]:
def fill_gaps(sequence, lookahead):
    """
    Function that replaces 0's with 1's in an array consisting of only 0's and 1's,
    if the gap between the next 1 is smaller than the lookahead.
    
    Example: 
        [0,1,0,0,0,1,0] as input with lookahead = 4
        outputs [0,1,1,1,1,1,0], the gap between 1's is filled        
    """
    
    i = 0
    while i < len(sequence) - lookahead:
        current = sequence[i]
        next = sequence[i + 1 : i + lookahead].tolist()
        
        if current and True in next:
            x = 0
            while not next[x]:
                sequence[i + 1 + x] = True
                x = x + 1
                
        i = i + 1

    return sequence

def to_time_string(seconds):
    """
    Given seconds in number, returns a string in the format hh:mm:ss (example: 01:30:45)
    """
    return str(datetime.timedelta(seconds=seconds))


def overlap(first_inter,second_inter):
    for f,s in ((first_inter,second_inter), (second_inter,first_inter)):
        #will check both ways
        for time in (f[0], f[1]):
            if s[0] <= time <= s[1]:
                return True
    else:
        return False
    
t1 = (0,10)
t2 = (7,15)
t3 = (15,20)
t4 = (50,60)

print(overlap(t1,t2))
print(overlap(t2,t3))
print(overlap(t3,t4))

def to_seconds(time):
    """
    Converts string of format hh:mm:ss to total number of seconds
    """
    if time == 'None':
        return -1
    try:
        hours = int(time.split(":")[0])
        minutes = int(time.split(":")[1])
        seconds = int(float(time.split(":")[2]))
        return hours*60*60 + minutes * 60 + seconds
    except:
#         print(time)
        if math.isnan(time):
            return -1



True
True
False


In [5]:
def get_bumper_timestamps_by_filename(filename, df):
    result = []
    row = df.loc[df['filename'] == filename].to_dict(orient='records')[0]
    if isinstance(row['start1'], str):
        result.append((row['start1'], row['end1']))
    if isinstance(row['start2'], str):
        result.append((row['start2'], row['end2']))
    if isinstance(row['start3'], str):
        result.append((row['start3'], row['end3']))
    if isinstance(row['start4'], str):
        result.append((row['start4'], row['end4']))
        
    result_final = []
    for start, end in result:
        result_final.append((to_seconds(start), to_seconds(end)))
    
    
    return result_final

annotations = pd.read_csv("annotations_bumpers.csv").dropna(how='all')
annotations.head()

get_bumper_timestamps_by_filename('214789B1_Expeditie_Robinson_S10_E2.mxf.mp4', annotations)

[(657, 671), (1981, 1993), (3104, 3118)]

In [6]:
episodes = [v for v in shows["Temptation_Island"]["S4"].values()]
episodes = episodes + [v for v in shows["Expeditie_Robinson"]["S10"].values()]
episodes = episodes + [v for v in shows["Expeditie_Robinson"]["S11"].values()]
episodes = episodes + [v for v in shows["Expeditie_Robinson"]["S13"].values()]
episodes = episodes + [v for v in shows["Expeditie_Robinson"]["S14"].values()]
episodes=sorted(episodes)
episodes

['214579A1_Expeditie_Robinson_S10_E1.mxf.mp4',
 '214789B1_Expeditie_Robinson_S10_E2.mxf.mp4',
 '214790A1_Expeditie_Robinson_S10_E3.mxf.mp4',
 '226557H1_Expeditie_Robinson_S11_E1.mxf.mp4',
 '226558H1_Expeditie_Robinson_S11_E2.mxf.mp4',
 '226559H1_Expeditie_Robinson_S11_E3.mxf.mp4',
 '250463H1_Expeditie_Robinson_S13_E2.mxf.mp4',
 '251218H1_Expeditie_Robinson_S13_E1.mxf.mp4',
 '260888H1_Expeditie_Robinson_S14_E3.mxf.mp4',
 '262135H1_Expeditie_Robinson_S14_E1.mxf.mp4',
 '262136H1_Expeditie_Robinson_S14_E2.mxf.mp4',
 '265707H1_Temptation_Island_S4_E4_high.mp4',
 '265708H1_Temptation_Island_S4_E5_high.mp4',
 '265709H1_Temptation_Island_S4_E6_high.mp4',
 '265710H1_Temptation_Island_S4_E7_high.mp4',
 '265711H1_Temptation_Island_S4_E8_high.mp4',
 '265712H1_Temptation_Island_S4_E9_high.mp4',
 '266841H1_Temptation_Island_S4_E1_high.mp4',
 '266843H1_Temptation_Island_S4_E2_high.mp4',
 '266844H1_Temptation_Island_S4_E3_high.mp4']

In [11]:
# vectors_dir = "./videos/full_histograms_binsize30_framejump3/"
# vectors_dir = "./videos_resized_w224/histogram_of_gradient_framejump3/"
# vectors_dir = "./videos_resized_w320/colortexturemoments_framejump3/"
vectors_dir = "./videos_resized_w320/color_histogram_binsize180_framejump3/"

vectors = [
#     "./videos_resized_w320/color_histogram_binsize180_framejump3/",
    "./videos_resized_w320/colortexturemoments_framejump3/",
    "./videos_resized_w224/histogram_of_gradient_framejump3/"
          ]

verbose = False

for dir in vectors:
    for ratio in [0.5, 0.6, 0.7, 0.8]:

        vector_files = [dir+e+'.p' for e in episodes]

        histograms = []
        lengths = []

        for f in vector_files:
            h = np.array(pickle.load(open(f, "rb")), np.float32)
            lengths.append(h.shape[0])
            histograms.append(h)

    #     print(histograms[0].shape)
        # histograms = np.vstack(histograms)
        # print(lengths)
        # histograms.shape
        total_overlap = 0
        total_detections = 0
        total_ground_truth = 0
        plot = False

        for episode_index in tqdm(range(0, len(episodes))):
        #     episode_index = ix
            if verbose:
                print("Detecting bumpers for: {}".format(episodes[episode_index]))
            histogram = histograms[episode_index]

            stepsize = 167
            s = 0
            e = stepsize
            result = np.empty(0)
            result_non_ratio = np.empty(0)

            for i in range(0, len(histogram) - 2*stepsize, stepsize):
                s = i
                e = i + stepsize
            #     print("start: {}, end: {}".format(s,e))
                q = histogram[s:e]
            #     rest = np.append(histogram[:s], histogram[e:], axis=0)
                rest = histogram[e + 3*stepsize:]

                vector_size = q.shape[1]
                # build the index, set vector size
            #     print("Building index with size: {}".format(rest.shape[0]))
                index = faiss.IndexFlatL2(vector_size)    
                # add vectors to the index
                index.add(rest)

                # we want to see k nearest neighbors
                k = 2
                # search with for matches with q
                scores, indexes = index.search(q, k)

                r = scores[:,0] / scores[:,1]
                r_nonratio = scores[:,0]

                result = np.concatenate((result, r))
                result_non_ratio = np.concatenate((result_non_ratio, r_nonratio))

            gap_fill = 150 

            # result_non_ratio = scipy.ndimage.median_filter(result_non_ratio, 10)
            if plot:
                plt.plot(result_non_ratio)
                plt.ylim(0,0.2)
                plt.show()

                # result = scipy.ndimage.median_filter(result, 10)
                plt.plot(result)
                plt.show()

                plt.plot(result < ratio)
                plt.show()

                plt.plot(fill_gaps(result < ratio, gap_fill))
                plt.show()

            below_threshold = fill_gaps(result < ratio, gap_fill)
            nonzeros = [[i for i,value in it] for key, it in itertools.groupby(enumerate(below_threshold), key=operator.itemgetter(1)) if key != 0]
            framejump = 3

            relevant = 0
            total = 0
            ground_truths = get_bumper_timestamps_by_filename(episodes[episode_index], annotations)

            for n in nonzeros:
                start = n[0]
                end = n[-1]
                if end - start > (2.5 * 25 / framejump): #only count when larger than x seconds 

                    total += 1
                    start_seconds = start / (25 / framejump)
                    end_seconds = end / (25 / framejump)

                    if verbose:
                        print(to_time_string(start_seconds))
                        print(to_time_string(end_seconds))

                    total_detections += 1
                    for gt in ground_truths:
                        if overlap(gt , (start_seconds, end_seconds)):
                            if verbose:
                                print("Overlap!")
                            relevant += 1
                            total_overlap += 1

            total_ground_truth += len(ground_truths)

            if verbose:
                if total > 0:
                    print("precision: \t{}".format(relevant / total))
                print(ground_truths)
                print("recall: \t\t{}".format(relevant / len(ground_truths)))
                print()
                print("======================================================================")

        total_precision = total_overlap / total_detections
        total_recall = total_overlap / total_ground_truth
        f1 = (2*total_precision*total_recall) / (total_precision + total_recall)

        print("Results {}, ratio = {}".format(dir, ratio))
        print("Total precision:\t {0:.3f}".format(total_precision))
        print("Total recall:\t {0:.3f}".format(total_recall))
#         print("F1:\t {0:.3f}".format(f1))

/mnt/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:68: RuntimeWarning: invalid value encountered in true_divide
/mnt/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:92: RuntimeWarning: invalid value encountered in less


Results ./videos_resized_w320/colortexturemoments_framejump3/, ratio = 0.5
Total precision:	 0.145
Total recall:	 0.542


Results ./videos_resized_w320/colortexturemoments_framejump3/, ratio = 0.6
Total precision:	 0.089
Total recall:	 0.667


Results ./videos_resized_w320/colortexturemoments_framejump3/, ratio = 0.7
Total precision:	 0.072
Total recall:	 0.833


Results ./videos_resized_w320/colortexturemoments_framejump3/, ratio = 0.8
Total precision:	 0.072
Total recall:	 0.958


Results ./videos_resized_w224/histogram_of_gradient_framejump3/, ratio = 0.5
Total precision:	 0.611
Total recall:	 0.917


Results ./videos_resized_w224/histogram_of_gradient_framejump3/, ratio = 0.6
Total precision:	 0.484
Total recall:	 0.938


Results ./videos_resized_w224/histogram_of_gradient_framejump3/, ratio = 0.7
Total precision:	 0.288
Total recall:	 0.979


Results ./videos_resized_w224/histogram_of_gradient_framejump3/, ratio = 0.8
Total precision:	 0.081
Total recall:	 0.979
